In [32]:
# encoding=utf-8
import os.path as osp
import os
import copy
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear
from sklearn.metrics import average_precision_score, roc_auc_score
from torch_geometric.data import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.datasets import ICEWS18
from torch_geometric.nn import TGNMemory, TransformerConv
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.models.tgn import (LastNeighborLoader, IdentityMessage, MeanAggregator,
                                           LastAggregator)
from torch_geometric import *
from torch_geometric.utils import negative_sampling
from tqdm import tqdm
import networkx as nx
import numpy as np
import math
import copy
import re
import time
import json
import pandas as pd
from random import choice
import gc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
# msg structure:    [src_node_feature,edge_attr,dst_node_feature]

# compute the best partition 
import datetime
# import community as community_louvain

import xxhash

# Find the edge index which the edge vector is corresponding to
def tensor_find(t,x):
    t_np=t.cpu().numpy()
    idx=np.argwhere(t_np==x)
    return idx[0][0]+1


def std(t):
    t = np.array(t)
    return np.std(t)


def var(t):
    t = np.array(t)
    return np.var(t)


def mean(t):
    t = np.array(t)
    return np.mean(t)

def hashgen(l):
    """Generate a single hash value from a list. @l is a list of
    string values, which can be properties of a node/edge. This
    function returns a single hashed integer value."""
    hasher = xxhash.xxh64()
    for e in l:
        hasher.update(e)
    return hasher.intdigest()


def cal_pos_edges_loss(link_pred_ratio):
    loss=[]
    for i in link_pred_ratio:
        loss.append(criterion(i,torch.ones(1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_multiclass(link_pred_ratio,labels):
    loss=[] 
    for i in range(len(link_pred_ratio)):
        loss.append(criterion(link_pred_ratio[i].reshape(1,-1),labels[i].reshape(-1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_autoencoder(decoded,msg):
    loss=[] 
    for i in range(len(decoded)):
        loss.append(criterion(decoded[i].reshape(1,-1),msg[i].reshape(-1)))
    return torch.tensor(loss)

In [2]:
%autosave 120  

Autosaving every 120 seconds


In [3]:
from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
def ns_time_to_datetime(ns):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00.000000000
    """
    dt = datetime.fromtimestamp(int(ns) // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def ns_time_to_datetime_US(ns):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00.000000000
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(ns) // 1000000000, tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def time_to_datetime_US(s):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(s), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')

    return s

def datetime_to_ns_time(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    return timeStamp

def datetime_to_ns_time_US(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp = timestamp.timestamp()
    timeStamp = timestamp * 1000000000
    return int(timeStamp)

def datetime_to_timestamp_US(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp = timestamp.timestamp()
    timeStamp = timestamp
    return int(timeStamp)





In [4]:
import psycopg2

from psycopg2 import extras as ex
connect = psycopg2.connect(database = 'tc_e5_cadets_dataset_db',
                           host = '/var/run/postgresql/',
                           user = 'postgres',
                           password = 'postgres',
                           port = '5432'
                          )


cur = connect.cursor()

In [6]:
graph_5_8=torch.load("./train_graph/graph_5_8.TemporalData.simple").to(device=device)
graph_5_9=torch.load("./train_graph/graph_5_9.TemporalData.simple").to(device=device)
graph_5_11=torch.load("./train_graph/graph_5_11.TemporalData.simple").to(device=device)


train_data=graph_5_8

In [7]:
# Constructing the map for nodeid to msg
sql="select * from node2id ORDER BY index_id;"
cur.execute(sql)
rows = cur.fetchall()

nodeid2msg={}  # nodeid => msg and node hash => nodeid
for i in rows:
    nodeid2msg[i[0]]=i[-1]
    nodeid2msg[i[-1]]={i[1]:i[2]}  

In [8]:
rel2id={1: 'EVENT_CLOSE',
 'EVENT_CLOSE': 1,
 2: 'EVENT_OPEN',
 'EVENT_OPEN': 2,
 3: 'EVENT_READ',
 'EVENT_READ': 3,
 4: 'EVENT_WRITE',
 'EVENT_WRITE': 4,
 5: 'EVENT_EXECUTE',
 'EVENT_EXECUTE': 5,
 6: 'EVENT_RECVFROM',
 'EVENT_RECVFROM': 6,
 7: 'EVENT_RECVMSG',
 'EVENT_RECVMSG': 7,
 8: 'EVENT_SENDMSG',
 'EVENT_SENDMSG': 8,
 9: 'EVENT_SENDTO',
 'EVENT_SENDTO': 9}

In [9]:
# train_data, val_data, test_data = data.train_val_test_split(val_ratio=0.15, test_ratio=0.15)
# max_node_num = max(torch.cat([data.dst,data.src]))+1
# max_node_num = data.num_nodes+1
max_node_num = 262626  # +1
# min_dst_idx, max_dst_idx = int(data.dst.min()), int(data.dst.max())
min_dst_idx, max_dst_idx = 0, max_node_num
neighbor_loader = LastNeighborLoader(max_node_num, size=20, device=device)

In [10]:
class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super(GraphAttentionEmbedding, self).__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(in_channels, out_channels, heads=8,
                                    dropout=0.0, edge_dim=edge_dim)
        self.conv2 = TransformerConv(out_channels*8, out_channels,heads=1, concat=False,
                             dropout=0.0, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        last_update.to(device)
        x = x.to(device)
        t = t.to(device)
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        x = F.relu(self.conv(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.lin_src = Linear(in_channels, in_channels*2)
        self.lin_dst = Linear(in_channels, in_channels*2)
        
        self.lin_seq = nn.Sequential(
            
            Linear(in_channels*4, in_channels*8),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels*8, in_channels*2),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels*2, int(in_channels//2)),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(int(in_channels//2), train_data.msg.shape[1]-32)                   
        )
        

    def forward(self, z_src, z_dst):
        h = torch.cat([self.lin_src(z_src) , self.lin_dst(z_dst)],dim=-1)      
         
        h = self.lin_seq (h)
        
        return h

memory_dim = 100         # node state
time_dim = 100
embedding_dim = 200      # edge embedding

memory = TGNMemory(
    max_node_num,
    train_data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(train_data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=train_data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=embedding_dim).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.00005, eps=1e-08,weight_decay=0.01)


# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()

# Helper vector to map global node indices to local ones.
assoc = torch.empty(max_node_num, dtype=torch.long, device=device)

saved_nodes=set()

In [11]:
BATCH=1024
def train(train_data):

    
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.
    saved_nodes=set()

    total_loss = 0
    
#     print("train_before_stage_data:",train_data)
    for batch in train_data.seq_batches(batch_size=BATCH):
        optimizer.zero_grad()

        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg        
        
        n_id = torch.cat([src, pos_dst]).unique()
#         n_id = torch.cat([src, pos_dst, neg_src, neg_dst]).unique()
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)
      
        z = gnn(z, last_update, edge_index, train_data.t[e_id], train_data.msg[e_id])
        
        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])       

        y_pred = torch.cat([pos_out], dim=0)
        
#         y_true = torch.cat([torch.zeros(pos_out.size(0),1),torch.ones(neg_out.size(0),1)], dim=0)#
        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l)           
          
        y_true = torch.tensor(y_true).to(device=device)
        y_true=y_true.reshape(-1).to(torch.long).to(device=device)
        
        loss = criterion(y_pred, y_true)
        
#         loss = criterion(pos_out, torch.ones_like(pos_out))
#         loss += criterion(neg_out, torch.zeros_like(neg_out))

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
#         for i in range(len(src)):
#             saved_nodes.add(int(src[i]))
#             saved_nodes.add(int(pos_dst[i]))

        loss.backward()
        optimizer.step()
        memory.detach()
#         print(z.shape)
        total_loss += float(loss) * batch.num_events
#     print("trained_stage_data:",train_data)
    return total_loss / train_data.num_events



In [12]:

train_graphs=[graph_5_8,graph_5_9,graph_5_11]

for epoch in tqdm(range(1, 31)):
    for g in train_graphs:
        loss = train(g)
        print(f'  Epoch: {epoch:02d}, Loss: {loss:.4f}')
#     scheduler.step()

model=[memory,gnn, link_pred,neighbor_loader]
os.system("mkdir -p ./models/")
torch.save(model,"./models/model_saved_share.pt")

  0%|                                                                                                   | 0/30 [00:00<?, ?it/s]

  Epoch: 01, Loss: 0.6433
  Epoch: 01, Loss: 0.2605


  3%|██▉                                                                                     | 1/30 [02:45<1:20:13, 165.97s/it]

  Epoch: 01, Loss: 0.3935
  Epoch: 02, Loss: 0.3017
  Epoch: 02, Loss: 0.2403


  7%|█████▊                                                                                  | 2/30 [05:32<1:17:29, 166.06s/it]

...

  Epoch: 28, Loss: 0.3745
  Epoch: 29, Loss: 0.2742
  Epoch: 29, Loss: 0.2250


 97%|████████████████████████████████████████████████████████████████████████████████████   | 29/30 [1:20:20<02:46, 166.26s/it]

  Epoch: 29, Loss: 0.3677
  Epoch: 30, Loss: 0.2737
  Epoch: 30, Loss: 0.2240


100%|███████████████████████████████████████████████████████████████████████████████████████| 30/30 [1:23:07<00:00, 166.25s/it]

  Epoch: 30, Loss: 0.3640


In [14]:
ls models/

model_saved_share.pt


# Generate the reconstruction results of every day

In [16]:
import time 

@torch.no_grad()
def test_day_new(inference_data,path):
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
    
    memory.eval()
    gnn.eval()
    link_pred.eval()
    
    memory.reset_state()  # Start with a fresh memory. 
    neighbor_loader.reset_state()  # Start with an empty graph.
    
    time_with_loss={} # key: time，  value： the losses
    total_loss = 0    
    edge_list=[]
    
    unique_nodes=torch.tensor([]).to(device=device)
    total_edges=0


    start_time=inference_data.t[0]
    event_count=0
    
    pos_o=[]
    
    loss_list=[]
    

    print("after merge:",inference_data)
    
    # Record the running time to evaluate the performance
    start = time.perf_counter()

    for batch in inference_data.seq_batches(batch_size=BATCH):
        
        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg
        unique_nodes=torch.cat([unique_nodes,src,pos_dst]).unique()
        total_edges+=BATCH
        
       
        n_id = torch.cat([src, pos_dst]).unique()       
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, inference_data.t[e_id], inference_data.msg[e_id])

        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])
        
        pos_o.append(pos_out)
        y_pred = torch.cat([pos_out], dim=0)
#         y_true = torch.cat(
#             [torch.ones(pos_out.size(0))], dim=0).to(torch.long)     
#         y_true=y_true.reshape(-1).to(torch.long)

        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l) 
        y_true = torch.tensor(y_true).to(device=device)
        y_true=y_true.reshape(-1).to(torch.long).to(device=device)

        # Only consider which edge hasn't been correctly predicted.
        # For benign graphs, the behaviors patterns are similar and therefore their losses are small
        # For anoamlous behaviors, some behaviors might not be seen before, so the probability of predicting those edges are low. Thus their losses are high.
        loss = criterion(y_pred, y_true)

        total_loss += float(loss) * batch.num_events
     
        
        # update the edges in the batch to the memory and neighbor_loader
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
        # compute the loss for each edge
        each_edge_loss= cal_pos_edges_loss_multiclass(pos_out,y_true)
        
        for i in range(len(pos_out)):
            srcnode=int(src[i])
            dstnode=int(pos_dst[i])  
            
            srcmsg=str(nodeid2msg[srcnode]) 
            dstmsg=str(nodeid2msg[dstnode])
            t_var=int(t[i])
            edgeindex=tensor_find(msg[i][16:-16],1)   
            edge_type=rel2id[edgeindex]
            loss=each_edge_loss[i]    

            temp_dic={}
            temp_dic['loss']=float(loss)
            temp_dic['srcnode']=srcnode
            temp_dic['dstnode']=dstnode
            temp_dic['srcmsg']=srcmsg
            temp_dic['dstmsg']=dstmsg
            temp_dic['edge_type']=edge_type
            temp_dic['time']=t_var
            

#             if "netflow" in srcmsg or "netflow" in dstmsg:
#                 temp_dic['loss']=0
            edge_list.append(temp_dic)
        
        event_count+=len(batch.src)
        if t[-1]>start_time+60000000000*15:
            # Here is a checkpoint, which records all edge losses in the current time window
#             loss=total_loss/event_count
            time_interval=ns_time_to_datetime_US(start_time)+"~"+ns_time_to_datetime_US(t[-1])

            end = time.perf_counter()
            time_with_loss[time_interval]={'loss':loss,
                                
                                          'nodes_count':len(unique_nodes),
                                          'total_edges':total_edges,
                                          'costed_time':(end-start)}
            
            
            log=open(path+"/"+time_interval+".txt",'w')
            
            for e in edge_list: 
#                 temp_key=e['srcmsg']+e['dstmsg']+e['edge_type']
#                 if temp_key in train_edge_set:      
# #                     e['loss']=(e['loss']-train_edge_set[temp_key]) if e['loss']>=train_edge_set[temp_key] else 0  
# #                     e['loss']=abs(e['loss']-train_edge_set[temp_key])
                    
#                     e['modified']=True
#                 else:
#                     e['modified']=False
                loss+=e['loss']

            loss=loss/event_count   
            print(f'Time: {time_interval}, Loss: {loss:.4f}, Nodes_count: {len(unique_nodes)}, Cost Time: {(end-start):.2f}s')
            edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True)   # Rank the results based on edge losses
            for e in edge_list: 
                log.write(str(e))
                log.write("\n") 
            event_count=0
            total_loss=0
            loss=0
            start_time=t[-1]
            log.close()
            edge_list.clear()
            
 
    return time_with_loss



In [15]:
graph_5_12=torch.load("./train_graph/graph_5_12.TemporalData.simple").to(device=device)
graph_5_15=torch.load("./train_graph/graph_5_15.TemporalData.simple").to(device=device)
graph_5_16=torch.load("./train_graph/graph_5_16.TemporalData.simple").to(device=device)
graph_5_17=torch.load("./train_graph/graph_5_17.TemporalData.simple").to(device=device)

In [ ]:
model=torch.load("./models/model_saved_share.pt")
memory,gnn, link_pred,neighbor_loader=model

In [17]:
ans_5_8=test_day_new(graph_5_8,"./graph_5_8")

after merge: TemporalData(dst=[1397931], msg=[1397931, 41], src=[1397931], t=[1397931])
Time: 2019-05-08 00:00:00.008989193~2019-05-08 00:15:12.819738250, Loss: 0.5694, Nodes_count: 956, Cost Time: 1.11s
Time: 2019-05-08 00:15:12.819738250~2019-05-08 00:34:12.708825175, Loss: 0.3083, Nodes_count: 1304, Cost Time: 1.79s
Time: 2019-05-08 00:34:12.708825175~2019-05-08 00:52:03.918713514, Loss: 0.5421, Nodes_count: 1655, Cost Time: 2.33s
Time: 2019-05-08 00:52:03.918713514~2019-05-08 01:07:12.359122777, Loss: 0.4147, Nodes_count: 2081, Cost Time: 2.98s
Time: 2019-05-08 01:07:12.359122777~2019-05-08 01:25:38.818821035, Loss: 0.2554, Nodes_count: 2276, Cost Time: 3.41s
Time: 2019-05-08 01:25:38.818821035~2019-05-08 01:41:28.558761306, Loss: 0.2894, Nodes_count: 2443, Cost Time: 3.88s
Time: 2019-05-08 01:41:28.558761306~2019-05-08 01:58:50.438624115, Loss: 0.3976, Nodes_count: 2721, Cost Time: 4.69s
Time: 2019-05-08 01:58:50.438624115~2019-05-08 02:14:03.438612575, Loss: 0.5095, Nodes_count: 

Time: 2019-05-08 21:56:50.555482977~2019-05-08 22:13:04.535675246, Loss: 0.0363, Nodes_count: 9830, Cost Time: 72.66s
Time: 2019-05-08 22:13:04.535675246~2019-05-08 22:28:22.555416040, Loss: 0.0343, Nodes_count: 9830, Cost Time: 73.07s
Time: 2019-05-08 22:28:22.555416040~2019-05-08 22:43:48.565468446, Loss: 0.0306, Nodes_count: 9830, Cost Time: 73.48s
Time: 2019-05-08 22:43:48.565468446~2019-05-08 22:59:12.555370285, Loss: 0.0281, Nodes_count: 9831, Cost Time: 73.89s
Time: 2019-05-08 22:59:12.555370285~2019-05-08 23:16:02.545423562, Loss: 0.2610, Nodes_count: 10591, Cost Time: 77.34s
Time: 2019-05-08 23:16:02.545423562~2019-05-08 23:32:10.545375783, Loss: 0.2203, Nodes_count: 11174, Cost Time: 78.64s
Time: 2019-05-08 23:32:10.545375783~2019-05-08 23:48:40.545322654, Loss: 0.2006, Nodes_count: 11466, Cost Time: 79.59s


In [18]:
ans_5_9=test_day_new(graph_5_9,"./graph_5_9")

after merge: TemporalData(dst=[7402525], msg=[7402525, 41], src=[7402525], t=[7402525])
Time: 2019-05-09 00:00:00.005370573~2019-05-09 00:16:10.545268695, Loss: 0.1944, Nodes_count: 34, Cost Time: 0.43s
Time: 2019-05-09 00:16:10.545268695~2019-05-09 00:31:30.555241471, Loss: 0.0307, Nodes_count: 34, Cost Time: 0.84s
Time: 2019-05-09 00:31:30.555241471~2019-05-09 00:46:46.555249187, Loss: 0.0377, Nodes_count: 35, Cost Time: 1.24s
Time: 2019-05-09 00:46:46.555249187~2019-05-09 01:01:56.555203078, Loss: 0.0426, Nodes_count: 37, Cost Time: 1.65s
Time: 2019-05-09 01:01:56.555203078~2019-05-09 01:18:22.555177589, Loss: 0.0345, Nodes_count: 38, Cost Time: 2.11s
Time: 2019-05-09 01:18:22.555177589~2019-05-09 01:33:38.555157961, Loss: 0.0384, Nodes_count: 39, Cost Time: 2.54s
Time: 2019-05-09 01:33:38.555157961~2019-05-09 01:48:52.555136318, Loss: 0.0378, Nodes_count: 39, Cost Time: 2.95s
Time: 2019-05-09 01:48:52.555136318~2019-05-09 02:03:56.555112888, Loss: 0.0500, Nodes_count: 39, Cost Time

Time: 2019-05-09 17:55:52.455519795~2019-05-09 18:12:16.474050413, Loss: 0.2436, Nodes_count: 10713, Cost Time: 377.74s
Time: 2019-05-09 18:12:16.474050413~2019-05-09 18:27:43.863827363, Loss: 0.4219, Nodes_count: 11032, Cost Time: 380.44s
Time: 2019-05-09 18:27:43.863827363~2019-05-09 18:43:09.954117708, Loss: 0.4606, Nodes_count: 11311, Cost Time: 381.86s
Time: 2019-05-09 18:43:09.954117708~2019-05-09 19:00:54.874007577, Loss: 0.6333, Nodes_count: 11578, Cost Time: 383.99s
Time: 2019-05-09 19:00:54.874007577~2019-05-09 19:16:12.306624651, Loss: 0.5619, Nodes_count: 11855, Cost Time: 385.95s
Time: 2019-05-09 19:16:12.306624651~2019-05-09 19:31:17.765397125, Loss: 0.3394, Nodes_count: 12154, Cost Time: 387.85s
Time: 2019-05-09 19:31:17.765397125~2019-05-09 19:46:24.404104182, Loss: 0.4282, Nodes_count: 12304, Cost Time: 389.35s
Time: 2019-05-09 19:46:24.404104182~2019-05-09 20:01:35.753820311, Loss: 0.4143, Nodes_count: 12547, Cost Time: 391.00s
Time: 2019-05-09 20:01:35.753820311~2019

In [19]:
ans_5_11=test_day_new(graph_5_11,"./graph_5_11")

after merge: TemporalData(dst=[1298877], msg=[1298877, 41], src=[1298877], t=[1298877])
Time: 2019-05-11 00:00:00.014349655~2019-05-11 00:15:10.923289891, Loss: 0.7230, Nodes_count: 1018, Cost Time: 0.74s
Time: 2019-05-11 00:15:10.923289891~2019-05-11 00:32:03.961727449, Loss: 0.5396, Nodes_count: 1728, Cost Time: 1.81s
Time: 2019-05-11 00:32:03.961727449~2019-05-11 00:48:59.682989652, Loss: 0.4649, Nodes_count: 2020, Cost Time: 2.83s
Time: 2019-05-11 00:48:59.682989652~2019-05-11 01:06:24.691531973, Loss: 0.3805, Nodes_count: 2418, Cost Time: 3.90s
Time: 2019-05-11 01:06:24.691531973~2019-05-11 01:21:50.262931075, Loss: 0.6917, Nodes_count: 2726, Cost Time: 6.12s
Time: 2019-05-11 01:21:50.262931075~2019-05-11 01:36:59.251529646, Loss: 0.2389, Nodes_count: 2989, Cost Time: 7.02s
Time: 2019-05-11 01:36:59.251529646~2019-05-11 01:52:29.101240073, Loss: 0.1717, Nodes_count: 3054, Cost Time: 8.93s
Time: 2019-05-11 01:52:29.101240073~2019-05-11 02:11:41.011246483, Loss: 0.6546, Nodes_count:

Time: 2019-05-11 23:02:31.751582100~2019-05-11 23:17:50.041154910, Loss: 0.4764, Nodes_count: 14801, Cost Time: 72.69s
Time: 2019-05-11 23:17:50.041154910~2019-05-11 23:32:58.392191613, Loss: 0.3470, Nodes_count: 15078, Cost Time: 73.99s
Time: 2019-05-11 23:32:58.392191613~2019-05-11 23:49:00.841202714, Loss: 0.6267, Nodes_count: 15232, Cost Time: 74.96s


In [20]:
ans_5_12=test_day_new(graph_5_12,"./graph_5_12")

after merge: TemporalData(dst=[1404800], msg=[1404800, 41], src=[1404800], t=[1404800])
Time: 2019-05-12 00:00:00.009704889~2019-05-12 00:15:44.829441975, Loss: 0.6369, Nodes_count: 850, Cost Time: 1.56s
Time: 2019-05-12 00:15:44.829441975~2019-05-12 00:31:07.179657269, Loss: 0.5119, Nodes_count: 1315, Cost Time: 2.43s
Time: 2019-05-12 00:31:07.179657269~2019-05-12 00:46:11.631021123, Loss: 0.7138, Nodes_count: 1645, Cost Time: 5.02s
Time: 2019-05-12 00:46:11.631021123~2019-05-12 01:01:22.279490585, Loss: 0.3149, Nodes_count: 2095, Cost Time: 6.27s
Time: 2019-05-12 01:01:22.279490585~2019-05-12 01:16:57.199370459, Loss: 0.6949, Nodes_count: 2393, Cost Time: 7.72s
Time: 2019-05-12 01:16:57.199370459~2019-05-12 01:34:43.379773300, Loss: 0.3989, Nodes_count: 2531, Cost Time: 8.25s
Time: 2019-05-12 01:34:43.379773300~2019-05-12 01:50:01.881070816, Loss: 0.5140, Nodes_count: 2882, Cost Time: 9.59s
Time: 2019-05-12 01:50:01.881070816~2019-05-12 02:05:21.991043789, Loss: 0.5319, Nodes_count: 

Time: 2019-05-12 22:15:24.049648957~2019-05-12 22:30:34.307913598, Loss: 0.3599, Nodes_count: 12539, Cost Time: 72.21s
Time: 2019-05-12 22:30:34.307913598~2019-05-12 22:45:52.147868084, Loss: 0.2700, Nodes_count: 12824, Cost Time: 73.15s
Time: 2019-05-12 22:45:52.147868084~2019-05-12 23:01:24.217632826, Loss: 0.6649, Nodes_count: 13009, Cost Time: 75.14s
Time: 2019-05-12 23:01:24.217632826~2019-05-12 23:16:34.389656674, Loss: 0.5139, Nodes_count: 13131, Cost Time: 76.80s
Time: 2019-05-12 23:16:34.389656674~2019-05-12 23:33:07.037830799, Loss: 0.6692, Nodes_count: 13840, Cost Time: 78.47s
Time: 2019-05-12 23:33:07.037830799~2019-05-12 23:48:57.527662974, Loss: 0.5240, Nodes_count: 14683, Cost Time: 80.10s


In [21]:
ans_5_15=test_day_new(graph_5_15,"./graph_5_15")

after merge: TemporalData(dst=[5841235], msg=[5841235, 41], src=[5841235], t=[5841235])
Time: 2019-05-15 00:00:00.006192320~2019-05-15 00:15:13.743767966, Loss: 0.4844, Nodes_count: 1709, Cost Time: 5.28s
Time: 2019-05-15 00:15:13.743767966~2019-05-15 00:30:15.346246304, Loss: 0.4370, Nodes_count: 2252, Cost Time: 8.96s
Time: 2019-05-15 00:30:15.346246304~2019-05-15 00:45:38.935375235, Loss: 0.4267, Nodes_count: 2810, Cost Time: 15.06s
Time: 2019-05-15 00:45:38.935375235~2019-05-15 01:00:49.986348448, Loss: 0.3025, Nodes_count: 3070, Cost Time: 18.67s
Time: 2019-05-15 01:00:49.986348448~2019-05-15 01:16:13.516335500, Loss: 0.3608, Nodes_count: 3402, Cost Time: 24.38s
Time: 2019-05-15 01:16:13.516335500~2019-05-15 01:31:15.876102508, Loss: 0.3794, Nodes_count: 3734, Cost Time: 30.00s
Time: 2019-05-15 01:31:15.876102508~2019-05-15 01:46:35.756168353, Loss: 0.3168, Nodes_count: 3885, Cost Time: 33.51s
Time: 2019-05-15 01:46:35.756168353~2019-05-15 02:02:03.445171413, Loss: 0.5027, Nodes_c

Time: 2019-05-15 17:59:50.764934674~2019-05-15 18:15:04.084749724, Loss: 0.5200, Nodes_count: 13181, Cost Time: 226.33s
Time: 2019-05-15 18:15:04.084749724~2019-05-15 18:30:12.692563554, Loss: 0.5208, Nodes_count: 13330, Cost Time: 229.22s
Time: 2019-05-15 18:30:12.692563554~2019-05-15 18:45:36.312255908, Loss: 0.4195, Nodes_count: 13647, Cost Time: 234.54s
Time: 2019-05-15 18:45:36.312255908~2019-05-15 19:00:43.932284310, Loss: 0.5164, Nodes_count: 13820, Cost Time: 239.27s
Time: 2019-05-15 19:00:43.932284310~2019-05-15 19:16:40.834769346, Loss: 0.4172, Nodes_count: 14111, Cost Time: 244.85s
Time: 2019-05-15 19:16:40.834769346~2019-05-15 19:32:22.054614132, Loss: 0.4489, Nodes_count: 14550, Cost Time: 251.23s
Time: 2019-05-15 19:32:22.054614132~2019-05-15 19:47:26.074698234, Loss: 0.6650, Nodes_count: 14698, Cost Time: 255.42s
Time: 2019-05-15 19:47:26.074698234~2019-05-15 20:02:51.452177789, Loss: 0.5192, Nodes_count: 14923, Cost Time: 259.31s
Time: 2019-05-15 20:02:51.452177789~2019

In [22]:
ans_5_16=test_day_new(graph_5_16,"./graph_5_16")

after merge: TemporalData(dst=[5285414], msg=[5285414, 41], src=[5285414], t=[5285414])
Time: 2019-05-16 00:00:00.014304910~2019-05-16 00:15:02.554575844, Loss: 0.6715, Nodes_count: 804, Cost Time: 2.05s
Time: 2019-05-16 00:15:02.554575844~2019-05-16 00:30:19.324454842, Loss: 0.4468, Nodes_count: 1482, Cost Time: 5.96s
Time: 2019-05-16 00:30:19.324454842~2019-05-16 00:45:31.114609140, Loss: 0.5222, Nodes_count: 1795, Cost Time: 9.19s
Time: 2019-05-16 00:45:31.114609140~2019-05-16 01:01:06.524611321, Loss: 0.3684, Nodes_count: 2285, Cost Time: 14.90s
Time: 2019-05-16 01:01:06.524611321~2019-05-16 01:17:54.074506572, Loss: 0.3526, Nodes_count: 2736, Cost Time: 17.08s
Time: 2019-05-16 01:17:54.074506572~2019-05-16 01:32:54.901798763, Loss: 0.3481, Nodes_count: 3005, Cost Time: 18.41s
Time: 2019-05-16 01:32:54.901798763~2019-05-16 01:48:08.801926929, Loss: 0.4402, Nodes_count: 3422, Cost Time: 20.29s
Time: 2019-05-16 01:48:08.801926929~2019-05-16 02:03:13.643316639, Loss: 0.4401, Nodes_cou

Time: 2019-05-16 17:43:27.572851052~2019-05-16 17:58:44.112324274, Loss: 0.6012, Nodes_count: 13962, Cost Time: 196.92s
Time: 2019-05-16 17:58:44.112324274~2019-05-16 18:14:14.082900415, Loss: 0.5144, Nodes_count: 14152, Cost Time: 199.97s
Time: 2019-05-16 18:14:14.082900415~2019-05-16 18:30:26.073055443, Loss: 0.4076, Nodes_count: 14362, Cost Time: 205.31s
Time: 2019-05-16 18:30:26.073055443~2019-05-16 18:45:49.432070287, Loss: 0.6121, Nodes_count: 14540, Cost Time: 211.26s
Time: 2019-05-16 18:45:49.432070287~2019-05-16 19:01:12.193027191, Loss: 0.4252, Nodes_count: 14758, Cost Time: 215.03s
Time: 2019-05-16 19:01:12.193027191~2019-05-16 19:16:22.673088266, Loss: 0.4956, Nodes_count: 14905, Cost Time: 219.03s
Time: 2019-05-16 19:16:22.673088266~2019-05-16 19:31:32.052688344, Loss: 0.5296, Nodes_count: 15236, Cost Time: 223.12s
Time: 2019-05-16 19:31:32.052688344~2019-05-16 19:46:40.442657625, Loss: 0.4727, Nodes_count: 15311, Cost Time: 225.76s
Time: 2019-05-16 19:46:40.442657625~2019

In [23]:
ans_5_17=test_day_new(graph_5_17,"./graph_5_17")

after merge: TemporalData(dst=[3912303], msg=[3912303, 41], src=[3912303], t=[3912303])
Time: 2019-05-17 00:00:00.011404169~2019-05-17 00:15:19.121412431, Loss: 0.1835, Nodes_count: 585, Cost Time: 3.87s
Time: 2019-05-17 00:15:19.121412431~2019-05-17 00:30:22.942273426, Loss: 0.4861, Nodes_count: 1547, Cost Time: 7.30s
Time: 2019-05-17 00:30:22.942273426~2019-05-17 00:45:36.772457892, Loss: 0.2546, Nodes_count: 1787, Cost Time: 11.62s
Time: 2019-05-17 00:45:36.772457892~2019-05-17 01:00:43.031312698, Loss: 0.7108, Nodes_count: 2101, Cost Time: 15.03s
Time: 2019-05-17 01:00:43.031312698~2019-05-17 01:15:44.002530354, Loss: 0.4704, Nodes_count: 2474, Cost Time: 19.18s
Time: 2019-05-17 01:15:44.002530354~2019-05-17 01:30:47.512391580, Loss: 0.5096, Nodes_count: 2866, Cost Time: 24.26s
Time: 2019-05-17 01:30:47.512391580~2019-05-17 01:46:06.532627237, Loss: 0.4898, Nodes_count: 2955, Cost Time: 26.52s
Time: 2019-05-17 01:46:06.532627237~2019-05-17 02:01:06.862180143, Loss: 0.3764, Nodes_co

# Initialize the node IDF

In [9]:

file_list=[]

file_path="graph_5_8/"
file_l=os.listdir("graph_5_8/")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_5_9/"
file_l=os.listdir("graph_5_9/")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_5_11/"
file_l=os.listdir("graph_5_11/")
for i in file_l:
    file_list.append(file_path+i)


file_path="graph_5_12/"
file_l=os.listdir("graph_5_12/")
for i in file_l:
    file_list.append(file_path+i)

# for f_path in tqdm(file_list):
#     f=open(f_path)
#     for line in f:
#         l=line.strip()
#         jdata=eval(l)
#         if jdata['loss']>0:
#             if 'netflow' not in str(jdata['srcmsg']):
#                 node_set.add(str(jdata['srcmsg']))
#             if 'netflow' not in str(jdata['dstmsg']):
#                 node_set.add(str(jdata['dstmsg'])) 

node_IDF={}
node_set = {}
for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']):
                if str(jdata['srcmsg']) not in node_set.keys():
                    node_set[str(jdata['srcmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['srcmsg'])].add(f_path)
            if 'netflow' not in str(jdata['dstmsg']):
                if str(jdata['dstmsg']) not in node_set.keys():
                    node_set[str(jdata['dstmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['dstmsg'])].add(f_path)
for n in node_set:
    include_count = len(node_set[n])   
    IDF=math.log(len(file_list)/(include_count+1))
    node_IDF[n] = IDF    


torch.save(node_IDF,"node_IDF")
print("IDF weight calculate complete!")


100%|████████████████████████████████████████████████████████████████████████████████████████| 315/315 [03:31<00:00,  1.49it/s]

IDF weight calculate complete!


In [4]:
def cal_train_IDF(find_str,file_list):
    include_count=0
    for f_path in (file_list):
        f=open(f_path)
        if find_str in f.read():
            include_count+=1             
    IDF=math.log(len(file_list)/(include_count+1))
    return IDF


def cal_IDF(find_str,file_path,file_list):
    file_list=os.listdir(file_path)
    include_count=0
    different_neighbor=set()
    for f_path in (file_list):
        f=open(file_path+f_path)
        if find_str in f.read():
            include_count+=1                
                
    IDF=math.log(len(file_list)/(include_count+1))
    
    return IDF,1

def cal_redundant(find_str,edge_list):
    
    different_neighbor=set()
    for e in edge_list:
        if find_str in str(e):
            different_neighbor.add(e[0])
            different_neighbor.add(e[1])
    return len(different_neighbor)-2

def cal_anomaly_loss(loss_list,edge_list,file_path):
    
    if len(loss_list)!=len(edge_list):
        print("error!")
        return 0
    count=0
    loss_sum=0
    loss_std=std(loss_list)
    loss_mean=mean(loss_list)
    edge_set=set()
    node_set=set()
    node2redundant={}
    
    thr=loss_mean+1.5*loss_std

    print("thr:",thr)

    for i in range(len(loss_list)):
        if loss_list[i]>thr:
            count+=1
            src_node=edge_list[i][0]
            dst_node=edge_list[i][1]
            
            loss_sum+=loss_list[i]
    
            node_set.add(src_node)
            node_set.add(dst_node)
            edge_set.add(edge_list[i][0]+edge_list[i][1])
    return count, loss_sum/count,node_set,edge_set
#     return count, count/len(loss_list)

# Construct the relations between time windows

In [12]:
# # node_IDF_5_9_12=torch.load("node_IDF_5_9-12")
# def cal_set_rel(s1,s2,file_list):
#     new_s=s1 & s2
#     count=0
#     for i in new_s:
# #     jdata=json.loads(i)
#         if 'netflow' not in i and '/home/admin/' not in i and  '/proc/' not in i :
        
# #         'netflow' not in i
# #         and 'usr' not in i and 'var' not in i
#             if i in node_IDF.keys():
#                 IDF=node_IDF[i]
#             else:
#                 IDF=math.log(len(file_list)/(1))
                
#             if i in node_IDF_4_4_7.keys():
#                 IDF4=node_IDF_4_4_7[i]
#             else:
#                 IDF4=math.log(len(file_list_4_4_7)/(1))    
            
# #             print(IDF)
#             if (IDF+IDF4)>9:
#                 print("node:",i," IDF:",IDF)
#                 count+=1
#     return count


# def cal_set_rel_bak(s1,s2,file_list):
#     new_s=s1 & s2
#     count=0
#     for i in new_s:
# #     jdata=json.loads(i)
#         if 'netflow' not in i \
#     and '/home/admin/' not in i \
#             and '/home/user/' not in i\
#             and '/tmp/' not in i\
#     and '/tmp/refload_pStageMem_log' not in i\
#             and 'com.' not in i:
        
# #         and '.dziauz.tinyflashlight' not in i \
# #             and '/data/system_ce/ not in i \
            
# #         and 'usr' not in i and 'proc' not in i and '675' not in i and 'firefox' not in i and 'tmp' not in i and 'thunderbird' not in i
# #         'netflow' not in i
# #         and 'usr' not in i and 'var' not in i
#             if i in node_IDF.keys():
#                 IDF=node_IDF[i]
#             else:
#                 IDF=math.log(len(file_list)/(1))           
                   
# #             print(IDF)
# #             print(len(file_list))
#             if IDF>math.log(len(file_list)*0.9/(1))  :
#                 print("node:",i," IDF:",IDF)
#                 count+=1
#     return count




def is_include_key_word_bak(s):
    keywords=[
         'netflow',

        '/home/admin/',
        '/home/user/',
         'proc',
        '/tmp/',
        '/var/spool/mqueue/',
        '/var/log/debug.log.0',
      
      ]
    flag=False
    for i in keywords:
        if i in s:
            flag=True
    return flag


def cal_set_rel_bak(s1,s2,file_list):
    new_s=s1 & s2
    count=0
    for i in new_s:
        if is_include_key_word_bak(i) is not True:
            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                IDF=math.log(len(file_list)/(1))           
                   
            if (IDF)>math.log(len(file_list)*0.9/(1))  :
                print("node:",i," IDF:",IDF)
                count+=1
    return count

In [6]:
pred_label={}   

filelist = os.listdir("./graph_5_15")
for f in filelist:

    pred_label["./graph_5_15/"+f]=0
    
filelist = os.listdir("./graph_5_16")
for f in filelist:

    pred_label["./graph_5_16/"+f]=0
    
filelist = os.listdir("./graph_5_17")
for f in filelist:
    pred_label["./graph_5_17/"+f]=0

In [7]:
pred_label

{'./graph_5_15/2019-05-15 10:03:41.465554650~2019-05-15 10:18:48.315349922.txt': 0,
 './graph_5_15/2019-05-15 15:11:29.355153391~2019-05-15 15:26:30.315069987.txt': 0,
 './graph_5_15/2019-05-15 14:40:58.314993755~2019-05-15 14:56:22.325270164.txt': 0,
 './graph_5_15/2019-05-15 14:56:22.325270164~2019-05-15 15:11:29.355153391.txt': 0,
 './graph_5_15/2019-05-15 23:04:18.584315206~2019-05-15 23:19:21.464426330.txt': 0,
 './graph_5_15/2019-05-15 21:03:34.932076311~2019-05-15 21:18:36.594847325.txt': 0,
 './graph_5_15/2019-05-15 00:15:13.743767966~2019-05-15 00:30:15.346246304.txt': 0,
 './graph_5_15/2019-05-15 00:45:38.935375235~2019-05-15 01:00:49.986348448.txt': 0,
 './graph_5_15/2019-05-15 18:30:12.692563554~2019-05-15 18:45:36.312255908.txt': 0,
 './graph_5_15/2019-05-15 07:39:16.336018368~2019-05-15 07:55:36.325862117.txt': 0,
 './graph_5_15/2019-05-15 10:34:18.902982618~2019-05-15 10:49:19.662777306.txt': 0,
 './graph_5_15/2019-05-15 08:46:56.325786951~2019-05-15 09:02:05.234725465.t

# Anomaly Detection 5-15

In [13]:
# node_IDF=torch.load("node_IDF_5_15")
node_IDF=torch.load("node_IDF")
y_data_5_15=[]
df_list_5_15=[]
# node_set_list=[]
history_list_5_15=[]
tw_que=[]
his_tw={}
current_tw={}
loss_list_5_15=[]

file_path_list=[]

file_path="./graph_5_15/"
file_l=os.listdir("./graph_5_15/")
for i in file_l:
    file_path_list.append(file_path+i)

index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    # Figure out which nodes are anomalous in this time window
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_5_15.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"./graph_5_15/")

    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    # Incrementally construct the queues
    added_que_flag=False
    for hq in history_list_5_15:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                # check if there are intersection between two time windows.
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_5_15.append(temp_hq)
    index_count+=1
    loss_list_5_15.append(loss_avg)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

index_count: 0
thr: 2.194360056990856
./graph_5_15/2019-05-15 00:00:00.006192320~2019-05-15 00:15:13.743767966.txt    3.7553629815613583  count: 8485  percentage: 0.08631388346354167  node count: 493  edge count: 451
index_count: 1
thr: 1.8967011079816516
./graph_5_15/2019-05-15 00:15:13.743767966~2019-05-15 00:30:15.346246304.txt    3.3338512660766  count: 5351  percentage: 0.08428364415322581  node count: 204  edge count: 197
index_count: 2
thr: 2.0548762933032765
./graph_5_15/2019-05-15 00:30:15.346246304~2019-05-15 00:45:38.935375235.txt    3.604907765926862  count: 9043  percentage: 0.08176902488425926  node count: 222  edge count: 222
index_count: 3
thr: 1.5347027361432513
./graph_5_15/2019-05-15 00:45:38.935375235~2019-05-15 01:00:49.986348448.txt    3.4026851467634915  count: 3159  percentage: 0.051416015625  node count: 148  edge count: 144
index_count: 4
thr: 1.811901356497414
./graph_5_15/2019-05-15 01:00:49.986348448~2019-05-15 01:16:13.516335500.txt    3.4977859843649006  

thr: 2.4039723503206485
./graph_5_15/2019-05-15 10:18:48.315349922~2019-05-15 10:34:18.902982618.txt    4.318013438905655  count: 6537  percentage: 0.08290635146103896  node count: 179  edge count: 172
index_count: 40
thr: 1.9027204455547244
./graph_5_15/2019-05-15 10:34:18.902982618~2019-05-15 10:49:19.662777306.txt    3.6041473408064464  count: 3676  percentage: 0.06526988636363637  node count: 140  edge count: 134
index_count: 41
thr: 2.3233825606719876
./graph_5_15/2019-05-15 10:49:19.662777306~2019-05-15 11:04:34.935316812.txt    4.2313182064819435  count: 7594  percentage: 0.07490924873737374  node count: 124  edge count: 119
index_count: 42
thr: 1.8074586093199594
./graph_5_15/2019-05-15 11:04:34.935316812~2019-05-15 11:20:14.345345440.txt    3.140305711104982  count: 6007  percentage: 0.08262268926056338  node count: 147  edge count: 140
index_count: 43
thr: 1.8356234974370471
./graph_5_15/2019-05-15 11:20:14.345345440~2019-05-15 11:35:14.525271329.txt    3.2895649384241703  co

thr: 2.1255446488607967
./graph_5_15/2019-05-15 20:02:51.452177789~2019-05-15 20:18:20.054640718.txt    3.9580907465360844  count: 2235  percentage: 0.0704070060483871  node count: 84  edge count: 79
index_count: 78
thr: 1.636668804430908
./graph_5_15/2019-05-15 20:18:20.054640718~2019-05-15 20:33:20.173943586.txt    3.5333224417396347  count: 4349  percentage: 0.057392842060810814  node count: 166  edge count: 156
index_count: 79
thr: 2.823731389306513
./graph_5_15/2019-05-15 20:33:20.173943586~2019-05-15 20:48:26.344519703.txt    4.345388863231559  count: 7176  percentage: 0.10617897727272728  node count: 154  edge count: 147
index_count: 80
thr: 2.35814581618105
./graph_5_15/2019-05-15 20:48:26.344519703~2019-05-15 21:03:34.932076311.txt    4.466750115694744  count: 6917  percentage: 0.07263314852150538  node count: 249  edge count: 245
index_count: 81
thr: 1.7247192530974131
./graph_5_15/2019-05-15 21:03:34.932076311~2019-05-15 21:18:36.594847325.txt    3.2876171714520477  count: 4

In [15]:

name_list=[]
for hl in history_list_5_15:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
#         for i in name_list:
#             pred_label[i]=1
        print(loss_count)

# Anoamly Detection 5-16

In [16]:
# node_IDF=torch.load("node_IDF_5_16")
# node_IDF=torch.load("node_IDF_5_9-12")
y_data_5_16=[]
df_list_5_16=[]
# node_set_list=[]
history_list_5_16=[]
tw_que=[]
his_tw={}
current_tw={}

file_path_list=[]

file_path="./graph_5_16/"
file_l=os.listdir("./graph_5_16/")
for i in file_l:
    file_path_list.append(file_path+i)

index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    # Figure out which nodes are anomalous in this time window
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_5_16.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"./graph_5_16/")

    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    # Incrementally construct the queues
    added_que_flag=False
    for hq in history_list_5_16:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                # check if there are intersection between two time windows.
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_5_16.append(temp_hq)
    index_count+=1
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

index_count: 0
thr: 2.725104913739848
./graph_5_16/2019-05-16 00:00:00.014304910~2019-05-16 00:15:02.554575844.txt    4.585362636863654  count: 3257  percentage: 0.08596389358108109  node count: 189  edge count: 145
index_count: 1
thr: 2.242179023565214
./graph_5_16/2019-05-16 00:15:02.554575844~2019-05-16 00:30:19.324454842.txt    4.432748329445932  count: 4066  percentage: 0.05754642210144927  node count: 213  edge count: 262
index_count: 2
thr: 2.3234012205822383
./graph_5_16/2019-05-16 00:30:19.324454842~2019-05-16 00:45:31.114609140.txt    3.9136210504051414  count: 4756  percentage: 0.08444602272727272  node count: 141  edge count: 134
index_count: 3
thr: 1.6817430690895092
./graph_5_16/2019-05-16 00:45:31.114609140~2019-05-16 01:01:06.524611321.txt    3.068151815172695  count: 7839  percentage: 0.07579478650990099  node count: 249  edge count: 242
index_count: 4
thr: 1.5668066564863836
./graph_5_16/2019-05-16 01:01:06.524611321~2019-05-16 01:17:54.074506572.txt    3.120767466474

thr: 2.9626802917298387
node: {'subject': 'php-fpm'}  IDF: 5.752572638825633
node: {'subject': 'nginx'}  IDF: 5.752572638825633
history queue: ./graph_5_16/2019-05-16 09:20:32.093582942~2019-05-16 09:36:08.903494477.txt
./graph_5_16/2019-05-16 09:51:22.110949680~2019-05-16 10:06:29.403713371.txt    4.688541164505954  count: 4956  percentage: 0.096796875  node count: 101  edge count: 92
index_count: 39
thr: 2.47509204989206
node: {'subject': 'nginx'}  IDF: 5.752572638825633
history queue: ./graph_5_16/2019-05-16 09:20:32.093582942~2019-05-16 09:36:08.903494477.txt
./graph_5_16/2019-05-16 10:06:29.403713371~2019-05-16 10:21:47.983513184.txt    4.389596619022277  count: 5522  percentage: 0.08559647817460317  node count: 183  edge count: 178
index_count: 40
thr: 1.8960156008413767
./graph_5_16/2019-05-16 10:21:47.983513184~2019-05-16 10:37:02.053456880.txt    3.966470134093636  count: 5141  percentage: 0.05906479779411765  node count: 179  edge count: 174
index_count: 41
thr: 2.71466932780

thr: 2.19545227887718
./graph_5_16/2019-05-16 19:16:22.673088266~2019-05-16 19:31:32.052688344.txt    3.7039780440214445  count: 5834  percentage: 0.08503381529850747  node count: 276  edge count: 273
index_count: 76
thr: 2.1282683455443965
./graph_5_16/2019-05-16 19:31:32.052688344~2019-05-16 19:46:40.442657625.txt    4.1483820859398115  count: 2964  percentage: 0.06578480113636363  node count: 86  edge count: 77
index_count: 77
thr: 1.7209776745274037
./graph_5_16/2019-05-16 19:46:40.442657625~2019-05-16 20:02:06.820142338.txt    3.0599066150336665  count: 3378  percentage: 0.07171365489130435  node count: 163  edge count: 168
index_count: 78
thr: 2.3245108315576517
./graph_5_16/2019-05-16 20:02:06.820142338~2019-05-16 20:17:11.412990296.txt    3.8753010489826125  count: 8057  percentage: 0.09366861979166667  node count: 310  edge count: 302
index_count: 79
thr: 1.8081077672077597
./graph_5_16/2019-05-16 20:17:11.412990296~2019-05-16 20:32:27.570220441.txt    3.969411948643162  count

In [19]:
name_list=[]
for hl in history_list_5_16:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
            print(i['name'])
#         print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

./graph_5_16/2019-05-16 09:20:32.093582942~2019-05-16 09:36:08.903494477.txt
./graph_5_16/2019-05-16 09:36:08.903494477~2019-05-16 09:51:22.110949680.txt
./graph_5_16/2019-05-16 09:51:22.110949680~2019-05-16 10:06:29.403713371.txt
./graph_5_16/2019-05-16 10:06:29.403713371~2019-05-16 10:21:47.983513184.txt
./graph_5_16/2019-05-16 20:32:27.570220441~2019-05-16 20:48:38.072848659.txt
./graph_5_16/2019-05-16 21:19:00.930018779~2019-05-16 21:34:46.231624861.txt
./graph_5_16/2019-05-16 21:34:46.231624861~2019-05-16 21:49:46.992678639.txt
./graph_5_16/2019-05-16 21:49:46.992678639~2019-05-16 22:06:14.950154813.txt
./graph_5_16/2019-05-16 22:06:14.950154813~2019-05-16 22:21:40.662702391.txt
./graph_5_16/2019-05-16 22:21:40.662702391~2019-05-16 22:36:45.602858389.txt
./graph_5_16/2019-05-16 22:36:45.602858389~2019-05-16 22:51:51.220035024.txt
./graph_5_16/2019-05-16 22:51:51.220035024~2019-05-16 23:07:16.890296254.txt
./graph_5_16/2019-05-16 23:07:16.890296254~2019-05-16 23:22:54.052353000.txt

# Anomaly Detection 5-17

In [20]:
# node_IDF=torch.load("node_IDF_5_17")
# node_IDF=torch.load("node_IDF_5_9-12")
y_data_5_17=[]
df_list_5_17=[]
# node_set_list=[]
history_list_5_17=[]
tw_que=[]
his_tw={}
current_tw={}

loss_list_5_17=[]

file_path_list=[]

file_path="./graph_5_17/"
file_l=os.listdir("./graph_5_17/")
for i in file_l:
    file_path_list.append(file_path+i)

index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_5_17.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"./clear_data/graph_5_17/")

    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_5_17:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])

                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_5_17.append(temp_hq)
    index_count+=1
    loss_list_5_17.append(loss_avg)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

index_count: 0
thr: 1.1795852483215774
./graph_5_17/2019-05-17 00:00:00.011404169~2019-05-17 00:15:19.121412431.txt    3.0619232276393005  count: 3086  percentage: 0.04305245535714286  node count: 268  edge count: 243
index_count: 1
thr: 2.0078692964972067
./graph_5_17/2019-05-17 00:15:19.121412431~2019-05-17 00:30:22.942273426.txt    3.4775138841842814  count: 4791  percentage: 0.08066742995689655  node count: 186  edge count: 177
index_count: 2
thr: 1.2057315811286506
./graph_5_17/2019-05-17 00:30:22.942273426~2019-05-17 00:45:36.772457892.txt    2.571637964544053  count: 4197  percentage: 0.0546484375  node count: 122  edge count: 119
index_count: 3
thr: 2.7981392522286805
./graph_5_17/2019-05-17 00:45:36.772457892~2019-05-17 01:00:43.031312698.txt    4.06124922693654  count: 7194  percentage: 0.12112742456896551  node count: 53  edge count: 48
index_count: 4
thr: 2.038159080075502
./graph_5_17/2019-05-17 01:00:43.031312698~2019-05-17 01:15:44.002530354.txt    3.528074391248113  cou

thr: 2.0365885415118274
./graph_5_17/2019-05-17 10:02:11.321524261~2019-05-17 10:17:26.881636687.txt    3.7463298083573537  count: 6049  percentage: 0.07876302083333334  node count: 152  edge count: 148
index_count: 40
thr: 2.5159587011102156
node: {'subject': 'nginx'}  IDF: 5.752572638825633
history queue: ./graph_5_17/2019-05-17 10:02:11.321524261~2019-05-17 10:17:26.881636687.txt
./graph_5_17/2019-05-17 10:17:26.881636687~2019-05-17 10:32:38.131495470.txt    4.285088405297908  count: 6435  percentage: 0.09107506793478261  node count: 246  edge count: 243
index_count: 41
thr: 1.5953392271406703
node: {'subject': 'nginx'}  IDF: 5.752572638825633
history queue: ./graph_5_17/2019-05-17 10:02:11.321524261~2019-05-17 10:17:26.881636687.txt
./graph_5_17/2019-05-17 10:32:38.131495470~2019-05-17 10:48:02.091564015.txt    3.1024392537701617  count: 1783  percentage: 0.06448929398148148  node count: 129  edge count: 128
index_count: 42
thr: 2.0637295854056323
./graph_5_17/2019-05-17 10:48:02.0

In [22]:
name_list=[]
for hl in history_list_5_17:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
            print(i['name'])
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

./graph_5_17/2019-05-17 10:02:11.321524261~2019-05-17 10:17:26.881636687.txt
./graph_5_17/2019-05-17 10:17:26.881636687~2019-05-17 10:32:38.131495470.txt
./graph_5_17/2019-05-17 10:32:38.131495470~2019-05-17 10:48:02.091564015.txt
102.90875594149468


In [23]:
from sklearn.metrics import average_precision_score, roc_auc_score

# from sklearn.metrics import plot_roc_curve,roc_curve,auc,roc_auc_score
import torch
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

def plot_thr():
    np.seterr(invalid='ignore')
    step=0.01
    thr_list=torch.arange(-5,5,step)
    
    

    precision_list=[]
    recall_list=[]
    fscore_list=[]
    accuracy_list=[]
    auc_val_list=[]
    for thr in thr_list:
        threshold=thr
        y_prediction=[]
        for i in y_test_scores:
            if i >threshold:
                y_prediction.append(1)
            else:
                y_prediction.append(0)
        precision,recall,fscore,accuracy,auc_val=classifier_evaluation(y_test, y_prediction)   
        precision_list.append(float(precision))
        recall_list.append(float(recall))
        fscore_list.append(float(fscore))
        accuracy_list.append(float(accuracy))
        auc_val_list.append(float(auc_val))

    max_fscore=max(fscore_list)
    max_fscore_index=fscore_list.index(max_fscore)
    print(max_fscore_index)
    print("max threshold:",thr_list[max_fscore_index])
    print('precision:',precision_list[max_fscore_index])
    print('recall:',recall_list[max_fscore_index])
    print('fscore:',fscore_list[max_fscore_index])
    print('accuracy:',accuracy_list[max_fscore_index])    
    print('auc:',auc_val_list[max_fscore_index])

    plt.plot(thr_list,precision_list,color='red',label='precision',linewidth=2.0,linestyle='-')
    plt.plot(thr_list,recall_list,color='orange',label='recall',linewidth=2.0,linestyle='solid')
    plt.plot(thr_list,fscore_list,color='y',label='F-score',linewidth=2.0,linestyle='dashed')
    plt.plot(thr_list,accuracy_list,color='g',label='accuracy',linewidth=2.0,linestyle='dashdot')
    plt.plot(thr_list,auc_val_list,color='b',label='auc_val',linewidth=2.0,linestyle='dotted')


    plt.xlabel("Threshold", fontdict={'size': 16})
    plt.ylabel("Rate", fontdict={'size': 16})
    plt.title("Different evaluation Indicators by varying threshold value", fontdict={'size': 12})
    plt.legend(loc='best', fontsize=12, markerscale=0.5)
    plt.show()

def classifier_evaluation(y_test, y_test_pred):
    # groundtruth, pred_value
    tn, fp, fn, tp =confusion_matrix(y_test, y_test_pred).ravel()
#     tn+=100
#     print(clf_name," : ")
    print('tn:',tn)
    print('fp:',fp)
    print('fn:',fn)
    print('tp:',tp)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    accuracy=(tp+tn)/(tp+tn+fp+fn)
    fscore=2*(precision*recall)/(precision+recall)    
    auc_val=roc_auc_score(y_test, y_test_pred)
    print("precision:",precision)
    print("recall:",recall)
    print("fscore:",fscore)
    print("accuracy:",accuracy)
    print("auc_val:",auc_val)
    return precision,recall,fscore,accuracy,auc_val

def minmax(data):
    min_val=min(data)
    max_val=max(data)
    ans=[]
    for i in data:
        ans.append((i-min_val)/(max_val-min_val))
    return ans



# label generation

In [25]:
labels={}

filelist = os.listdir("graph_5_15")
for f in filelist:
    labels["./graph_5_15/"+f]=0

    
filelist = os.listdir("graph_5_16")
for f in filelist:
    labels["./graph_5_16/"+f]=0

    
filelist = os.listdir("graph_5_17")
for f in filelist:
    labels["./graph_5_17/"+f]=0


In [26]:
attack_list=[
    './graph_5_16/2019-05-16 09:20:32.093582942~2019-05-16 09:36:08.903494477.txt', 
 './graph_5_16/2019-05-16 09:36:08.903494477~2019-05-16 09:51:22.110949680.txt', 
 './graph_5_16/2019-05-16 09:51:22.110949680~2019-05-16 10:06:29.403713371.txt', 
 './graph_5_16/2019-05-16 10:06:29.403713371~2019-05-16 10:21:47.983513184.txt', 

# Here are the "fake" FP time windows described in Section 5.2 in the paper.
 './graph_5_16/2019-05-16 20:32:27.570220441~2019-05-16 20:48:38.072848659.txt', 
 './graph_5_16/2019-05-16 21:19:00.930018779~2019-05-16 21:34:46.231624861.txt', 
 './graph_5_16/2019-05-16 21:34:46.231624861~2019-05-16 21:49:46.992678639.txt', 
 './graph_5_16/2019-05-16 21:49:46.992678639~2019-05-16 22:06:14.950154813.txt', 
 './graph_5_16/2019-05-16 22:06:14.950154813~2019-05-16 22:21:40.662702391.txt', 
 './graph_5_16/2019-05-16 22:21:40.662702391~2019-05-16 22:36:45.602858389.txt', 
 './graph_5_16/2019-05-16 22:36:45.602858389~2019-05-16 22:51:51.220035024.txt', 
 './graph_5_16/2019-05-16 22:51:51.220035024~2019-05-16 23:07:16.890296254.txt', 
 './graph_5_16/2019-05-16 23:07:16.890296254~2019-05-16 23:22:54.052353000.txt',

    
    './graph_5_17/2019-05-17 10:02:11.321524261~2019-05-17 10:17:26.881636687.txt', 
 './graph_5_17/2019-05-17 10:17:26.881636687~2019-05-17 10:32:38.131495470.txt', 
 './graph_5_17/2019-05-17 10:32:38.131495470~2019-05-17 10:48:02.091564015.txt'
]


# 结合 Rao的文档，对GT文档进行补充。检测出的准确率非常高。

for i in attack_list:
    labels[i]=1

In [61]:
labels

{'./graph_5_15/2019-05-15 10:03:41.465554650~2019-05-15 10:18:48.315349922.txt': 0,
 './graph_5_15/2019-05-15 15:11:29.355153391~2019-05-15 15:26:30.315069987.txt': 0,
 './graph_5_15/2019-05-15 14:40:58.314993755~2019-05-15 14:56:22.325270164.txt': 0,
 './graph_5_15/2019-05-15 14:56:22.325270164~2019-05-15 15:11:29.355153391.txt': 0,
 './graph_5_15/2019-05-15 23:04:18.584315206~2019-05-15 23:19:21.464426330.txt': 0,
 './graph_5_15/2019-05-15 21:03:34.932076311~2019-05-15 21:18:36.594847325.txt': 0,
 './graph_5_15/2019-05-15 00:15:13.743767966~2019-05-15 00:30:15.346246304.txt': 0,
 './graph_5_15/2019-05-15 00:45:38.935375235~2019-05-15 01:00:49.986348448.txt': 0,
 './graph_5_15/2019-05-15 18:30:12.692563554~2019-05-15 18:45:36.312255908.txt': 0,
 './graph_5_15/2019-05-15 07:39:16.336018368~2019-05-15 07:55:36.325862117.txt': 0,
 './graph_5_15/2019-05-15 10:34:18.902982618~2019-05-15 10:49:19.662777306.txt': 0,
 './graph_5_15/2019-05-15 08:46:56.325786951~2019-05-15 09:02:05.234725465.t

In [27]:
y=[]
y_pred=[]
for i in labels:
    y.append(labels[i])
    y_pred.append(pred_label[i])

In [28]:
classifier_evaluation(y,y_pred)

tn: 238
fp: 0
fn: 0
tp: 16
precision: 1.0
recall: 1.0
fscore: 1.0
accuracy: 1.0
auc_val: 1.0


(1.0, 1.0, 1.0, 1.0, 1.0)

# Count the number of the attack edges

In [1]:
def keyword_hit(line):
    attack_nodes=[
           'nginx',
#         '128.55.12.167',
#          '4.21.51.250',
#          'ocMain.py',
        'python',
#          '98.23.182.25',
#         '108.192.100.31',
        'hostname',
        'whoami',
#         'cat /etc/passwd',  
        ]
    flag=False
    for i in attack_nodes:
        if i in line:
            flag=True
            break
    return flag



files=[
    'graph_5_16/2019-05-16 09:20:32.093582942~2019-05-16 09:36:08.903494477.txt', 
 'graph_5_16/2019-05-16 09:36:08.903494477~2019-05-16 09:51:22.110949680.txt', 
 'graph_5_16/2019-05-16 09:51:22.110949680~2019-05-16 10:06:29.403713371.txt', 
 'graph_5_16/2019-05-16 10:06:29.403713371~2019-05-16 10:21:47.983513184.txt', 
    'graph_5_17/2019-05-17 10:02:11.321524261~2019-05-17 10:17:26.881636687.txt', 
 'graph_5_17/2019-05-17 10:17:26.881636687~2019-05-17 10:32:38.131495470.txt', 
 'graph_5_17/2019-05-17 10:32:38.131495470~2019-05-17 10:48:02.091564015.txt'
]


In [4]:
attack_edge_count=0
for fpath in tqdm(files):
    f=open(fpath)
    for line in f:
        if keyword_hit(line):
            attack_edge_count+=1
print(attack_edge_count)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 41.25it/s]

793


# Visualization

In [49]:
import os

from graphviz import Digraph
import networkx as nx
import datetime
import community.community_louvain as community_louvain
from tqdm import tqdm



# Some common path abstraction for visualization
replace_dic = {
    '/run/shm/':'/run/shm/*',
    #     '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/':'/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/*',
    '/home/admin/.cache/mozilla/firefox/':'/home/admin/.cache/mozilla/firefox/*',
    '/home/admin/.mozilla/firefox':'/home/admin/.mozilla/firefox*',
    '/data/replay_logdb/':'/data/replay_logdb/*',
    '/home/admin/.local/share/applications/':'/home/admin/.local/share/applications/*',

    '/usr/share/applications/':'/usr/share/applications/*',
    '/lib/x86_64-linux-gnu/':'/lib/x86_64-linux-gnu/*',
    '/proc/':'/proc/*',
    '/stat':'*/stat',
    '/etc/bash_completion.d/':'/etc/bash_completion.d/*',
    '/usr/bin/python2.7':'/usr/bin/python2.7/*',
    '/usr/lib/python2.7':'/usr/lib/python2.7/*',
    '/data/data/org.mozilla.fennec_firefox_dev/cache/':'/data/data/org.mozilla.fennec_firefox_dev/cache/*',
    'UNNAMED':'UNNAMED *',
    '/usr/ports/':'/usr/ports/*',
    '/usr/home/user/test':'/usr/home/user/test/*',
    '/tmp//':'/tmp//*',
    '/home/admin/backup/':'/home/admin/backup/*',
    '/home/admin/./backup/':'/home/admin/./backup/*',
    '/usr/home/admin/./test/':'/usr/home/admin/./test/*',
    '/usr/home/admin/test/':'/usr/home/admin/test/*',
    '/home/admin/out':'/home/admin/out*',
}


def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    return path_name


# Users should manually put the detected anomalous time windows here
attack_list = [
    'graph_5_16/2019-05-16 09:20:32.093582942~2019-05-16 09:36:08.903494477.txt', 
 'graph_5_16/2019-05-16 09:36:08.903494477~2019-05-16 09:51:22.110949680.txt', 
 'graph_5_16/2019-05-16 09:51:22.110949680~2019-05-16 10:06:29.403713371.txt', 
 'graph_5_16/2019-05-16 10:06:29.403713371~2019-05-16 10:21:47.983513184.txt', 
    'graph_5_17/2019-05-17 10:02:11.321524261~2019-05-17 10:17:26.881636687.txt', 
 'graph_5_17/2019-05-17 10:17:26.881636687~2019-05-17 10:32:38.131495470.txt', 
 'graph_5_17/2019-05-17 10:32:38.131495470~2019-05-17 10:48:02.091564015.txt'
]

original_edges_count = 0
graphs = []
gg = nx.DiGraph()
count = 0
for path in tqdm(attack_list):
    if ".txt" in path:
        line_count = 0
        node_set = set()
        tempg = nx.DiGraph()
        f = open(path, "r")
        edge_list = []
        for line in f:
            count += 1
            l = line.strip()
            jdata = eval(l)
            edge_list.append(jdata)

        edge_list = sorted(edge_list, key=lambda x: x['loss'], reverse=True)
        original_edges_count += len(edge_list)

        loss_list = []
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean = mean(loss_list)
        loss_std = std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr = loss_mean + 1.5 * loss_std
        print("thr:", thr)
        for e in edge_list:
            if e['loss'] > thr:
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),
                               str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))), str(hashgen(replace_path_name(e['dstmsg']))),
                            loss=e['loss'], srcmsg=e['srcmsg'], dstmsg=e['dstmsg'], edge_type=e['edge_type'],
                            time=e['time'])


partition = community_louvain.best_partition(gg.to_undirected())

# Generate the candidate subgraphs based on community discovery results
communities = {}
max_partition = 0
for i in partition:
    if partition[i] > max_partition:
        max_partition = partition[i]
for i in range(max_partition + 1):
    communities[i] = nx.DiGraph()
for e in gg.edges:
    communities[partition[e[0]]].add_edge(e[0], e[1])
    communities[partition[e[1]]].add_edge(e[0], e[1])


# Define the attack nodes. They are **only be used to plot the colors of attack nodes and edges**.
# They won't change the detection results.
def attack_edge_flag(msg):
    attack_edge_type = [
        "'nginx'",
        "'cat'",
        "'scp'",
        "'find'",
        "'bash'",
        "/etc/passwd",
        "/usr/home/user/",
        "128.55.12.167",
        "4.21.51.250",
        "128.55.12.233",
    ]
    flag = False
    for i in attack_edge_type:
        if i in str(msg):
            flag = True
    return flag


# Plot and render candidate subgraph
os.system(f"mkdir -p ./graph_visual/")
graph_index = 0
for c in communities:
    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'

    for e in communities[c].edges:
        try:
            temp_edge = gg.edges[e]
            srcnode = e['srcnode']
            dstnode = e['dstnode']
        except:
            pass

        if True:
            # source node
            if "'subject': '" in temp_edge['srcmsg']:
                src_shape = 'box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape = 'oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape = 'diamond'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color = 'red'
            else:
                src_node_color = 'blue'
            dot.node(name=str(hashgen(replace_path_name(temp_edge['srcmsg']))), label=str(
                replace_path_name(temp_edge['srcmsg']) + str(
                    partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), color=src_node_color,
                     shape=src_shape)

            # destination node
            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape = 'box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape = 'oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape = 'diamond'
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color = 'red'
            else:
                dst_node_color = 'blue'
            dot.node(name=str(hashgen(replace_path_name(temp_edge['dstmsg']))), label=str(
                replace_path_name(temp_edge['dstmsg']) + str(
                    partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), color=dst_node_color,
                     shape=dst_shape)

            if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
                edge_color = 'red'
            else:
                edge_color = 'blue'
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),
                     str(hashgen(replace_path_name(temp_edge['dstmsg']))), label=temp_edge['edge_type'],
                     color=edge_color)

    dot.render(f'./graph_visual/subgraph_' + str(graph_index), view=False)
    graph_index += 1





 14%|█████████████▏                                                                              | 1/7 [00:00<00:04,  1.45it/s]

0.4508893859632366
1.0734734735406513
thr: 2.0610995962742136


 29%|██████████████████████████▎                                                                 | 2/7 [00:01<00:02,  1.82it/s]

0.37147385526655363
0.8381559447774005
thr: 1.6287077724326542


 43%|███████████████████████████████████████▍                                                    | 3/7 [00:01<00:02,  1.89it/s]

...

0.40583913196470994
1.0871662730314116
thr: 2.0365885415118274


 86%|██████████████████████████████████████████████████████████████████████████████▊             | 6/7 [00:03<00:00,  1.53it/s]

0.5502357208182536
1.310481986861308
thr: 2.5159587011102156


100%|████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:03<00:00,  1.76it/s]

0.3727924665600163
0.815031173720436
thr: 1.5953392271406703
